In [ ]:
# general imports
import warnings
import numpy as np
import os
import pandas as pd
import scipy as sp
from scipy.sparse import coo_matrix
import collections
import scanpy as sc


# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.manifold import TSNE
import sklearn as sk

# matplotlib settings for Jupyter notebooks only
%matplotlib inline

import pickle
import gzip
from pathlib import Path

aug_data_path = "/Users/ivicha/Documents/GreeneLab/NormalCortex_Data/From_brainMap/matrix/results/pseudos"
cybersort_path = "/Users/ivicha/Documents/GreeneLab/NormalCortex_Data/From_brainMap/matrix/results/pseudos"
data_path = "/Users/ivicha/Documents/GreeneLab/NormalCortex_Data/From_brainMap/matrix"
scpred_path = "/Users/ivicha/Documents/GreeneLab/NormalCortex_Data/From_brainMap/metafolder"

In [ ]:
# 6k process

# read in the data

os.chdir(data_path)
adata = sc.read_csv('matrix.csv')
adata.var_names_make_unique()  # this is unnecessary if using `var_names='gene_ids'` in `sc.read_10x_mtx`
adata.var['gene_ids'] = adata.var_names

In [ ]:

# add metadata

os.chdir(scpred_path)
meta_data = pd.read_csv("metadata.csv", sep="\,")

#oligos
meta_data = meta_data.replace(to_replace=['Oligo L4-6 OPALIN', 'Oligo L4-6 MOBP COL18A1'],value=['Oligodendrocyte', 'Oligodendrocyte'])
#microglia
meta_data = meta_data.replace(to_replace=['Micro L1-6 C1QC'], value=['Microglia'])

#Astrocyes
meta_data = meta_data.replace(to_replace=['Astro L1-6 FGFR3 ETNPPL', 'Astro L1 FGFR3 MT1G','Astro L1 FGFR3 FOS'], value=['Astrocyte', 'Astrocyte', 'Astrocyte'])

#opcs
meta_data = meta_data.replace(to_replace=['OPC L1-6 MYT1'], value= ['OPC'])

#endocytes
meta_data = meta_data.replace(to_replace=['Endo L2-5 CLDN5'], value=['Endocyte'])

#others
meta_data = meta_data.fillna('')
meta_data = meta_data.replace(to_replace=['VLMC L1-3 CYP1B1', 'Peri L1-6 MUSTN1', 'NA', ''], value= ['Other', 'Other', 'Other', 'Other'])

meta_data = meta_data.replace(regex=r'Inh', value= 'Neuron')
meta_data = meta_data.replace(regex=r'Exc', value = 'Neuron')

meta_data = meta_data.replace(regex='Neuron\s\w\w\s\w*\w*', value = 'Neuron')
meta_data = meta_data.replace(regex='Neuron\s\w\w\W\s\w*\w*', value = 'Neuron')

meta_data = meta_data.replace(regex='Neuron\s\w*', value = 'Neuron')

meta_data = meta_data.replace(regex='Neuron\W\w\s\w*', value = 'Neuron')

meta_data = meta_data.replace(regex='Neuron\s\w*', value = 'Neuron')

meta_data['scpred_prediction']= meta_data.cell_type_alias_label

meta_data.cell_type_alias_label = meta_data.cell_type_alias_label.astype('category')

os.chir(aug_data_path)

#barcodes = pd.read_csv(f"{data_path}/barcodes.tsv", header=None, names=['code'])
#meta_df = barcodes.join(other=meta_data, on=['code'], how='left', sort=False)
adata.obs['scpred_CellType'] = meta_data.cell_type_alias_label.values
adata.obs['CellType'] = meta_data.cell_type_alias_label.values


# filter out cells with less than 200 genes and genes expressed in less than 3 cells
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

# remove genes with high mitochondrial content
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

# slice the data based on the plots from above
# remove cells with more than 3500 genes
# remove cells with more than 10% MTgenes
adata = adata[adata.obs.n_genes_by_counts < 2000, :]
adata = adata[adata.obs.pct_counts_mt < 10, :]


# normalize to 10K counts per cell
sc.pp.normalize_total(adata, target_sum=1e4)

# log data
sc.pp.log1p(adata)

# get high variance genes
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
pbmc_6k_gene_ids = adata.var['gene_ids']

# and filter
adata = adata[:, adata.var.highly_variable]

pbmc_6k_gene_ids_variable = adata.var['gene_ids']

In [ ]:
pbmc_6k_gene_ids_variable = adata.var['gene_ids']

In [ ]:
genes_all = adata.var['gene_ids']

genes_all = list(set.intersection(*[set(x) for x in genes_all]))
len(genes_all)

In [ ]:
genes_all_variable = adata.var['gene_ids']

genes_all_variable = list(set.intersection(*[set(x) for x in genes_all_variable]))
len(genes_all_variable)

In [ ]:
# write out the gene ids
gene_out_file = os.path.join(aug_data_path, "intersection_genes.pkl")
gene_out_path = Path(gene_out_file)
pickle.dump( genes_all, open( gene_out_path, "wb" ) )